In [1]:
import os
os.environ["TAVILY_API_KEY"] ="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"

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=5)
search_results = search.invoke("what is the price of AXA stock? Please return the price of the stock today at the Paris place.")


In [4]:
search_results

[{'url': 'https://www.gurufocus.com/stock/FRA:AXA/summary?mobile',
  'content': "What is AXA SA(FRA:AXA)'s stock price today? The current price of FRA:AXA is €36.69. The 52 week high of FRA:AXA is €36.90 and 52 week low is €"},
 {'url': 'https://www.marketwatch.com/investing/stock/cs?countrycode=fr',
  'content': 'Key Data. Open €36.91; Day Range 36.72 - 37.01; 52 Week Range 29.04 - 37.14; Market Cap €80.88B; Shares Outstanding 2.21B; Public Float 2.03B; Beta N/A'},
 {'url': 'https://live.euronext.com/en/product/equities/FR0000120628-XPAR',
  'content': 'AXA. Euronext Paris FR0000120628 - Stock. CAC 40 -1.20% SBF 120 -1.19% EUR / USD -0.29% EUR / GBP -0.14%. € 36.37. Last traded price. 03/02/2025 - 17:35'},
 {'url': 'https://www.nasdaq.com/market-activity/stocks/axahy',
  'content': "Key Data ; Exchange. Other OTC ; Today's High/Low. $37.5400/$36.9940 ; Share Volume. 67,984 ; Previous Close. $37.84 ; Annualized Dividend. $2.102958."},
 {'url': 'https://www.cnbc.com/quotes/CS-FR',
  'co

In [5]:
from langchain.tools import tool
from pydantic import BaseModel, Field
import numpy as np

class CalcInput(BaseModel):
    a: float = Field(description = "first number")
    b: float = Field(description = "second number")
@tool(parse_docstring=True)
def gog(a:float ,b: float)-> float:
    """A function doing complex mathematics taking to floats as input and outputing another float

    Args:
        a (float): an input float
        b (float): a second input float

    Returns:
        float: the result of the operation
    """

    return np.log(np.exp(a))*b +2


from langchain_core.tools import StructuredTool

Gog_tool = StructuredTool.from_function(func=gog,
                                        name = "Function doing complex operations",
                                        description = "The function allows you to do a gog which is a special operations on two floats",
                                        args_schema=CalcInput,
                                        return_direct=True)

In [6]:
from langchain.tools import tool
from pydantic import BaseModel, Field
import numpy as np

class BuyInput(BaseModel):
    k: float = Field(description = "number of shares to buy")
    s: str = Field(description="the name of the stock to buy")
    
@tool(parse_docstring=True)
def buy(k:int , s : str)-> str:
    """A function buying k number of stock s

    Args:
        k (int): number of shares to buy
        s (str): the name of the stock to buy

    Returns:
        str: the result of the operation
    """
    print(f"bought {k} shares of {s}")
    return f"bought {k} shares of {s}"


from langchain_core.tools import StructuredTool

Buy_tool = StructuredTool.from_function(func=buy,
                                        name = "Function buying stocks",
                                        description = "The function allows to buy stocks and returns adescription of the operation",
                                        args_schema=BuyInput,
                                        return_direct=True)

In [ ]:
from transformers import AutoTokenizer , AutoModelForCausalLM , pipeline
import ctransformers as CT

llm = CT.AutoModelForCausalLM.from_pretrained('./LLMs/Mistralv1.02',context_length = 5000 , hf = True)

tokenizer = AutoTokenizer.from_pretrained('./LLMs/Mistralv1.02/tokenizer')

from langchain_huggingface.llms import HuggingFacePipeline

pipe = pipeline('text-generation', model=llm, tokenizer=tokenizer,max_new_tokens = 5000 ,device="cuda")
hf = HuggingFacePipeline(pipeline=pipe)

CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Device set to use cuda


In [9]:
import torch
torch.cuda.is_available()

True

In [10]:
from functools import reduce
from operator import add

reduce(add,["1","2"])

'12'

In [11]:
from langchain_core.messages import SystemMessage
from langchain_huggingface import ChatHuggingFace
Ch = ChatHuggingFace(llm = hf)
Ch_with_tools = Ch.bind_tools([Gog_tool , search , Buy_tool])
langgraph_ex = create_react_agent(Ch_with_tools, tools = [Gog_tool,search,Buy_tool] , debug=True)

In [12]:
query = "use the Gog tool to find the result evaluated at 2 and 3?"
inputs = {"messages" : [('user',query)]}

In [13]:
Ch_with_tools.invoke(query)

c:\Users\benya\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


AIMessage(content='<s> [INST] use the Gog tool to find the result evaluated at 2 and 3? [/INST] I\'m assuming you\'re asking about using the GOG (GNU Online Calculator) tool to evaluate expressions. GOG is a free, web-based calculator that can evaluate mathematical expressions. However, I cannot directly use GOG in this text-based conversation. Instead, I can guide you on how to use it.\n\n1. Open your web browser and go to the GOG calculator website: https://www.functionx.com/calculator/calculator.aspx\n2. In the input box, type the expression you want to evaluate, which in this case is `f(x)` with `x` being 2 and 3. Since we don\'t have a specific function `f(x)` defined, let\'s assume it\'s a simple function like `x^2` for this example. So, the expression would be `2^2` and `3^2`.\n3. Press the "Calculate" button to get the results.\n4. The results will be displayed on the screen. For `2^2`, the result is 4, and for `3^2`, the result is 9.\n\nSo, the results evaluated at 2 and 3 usi

In [14]:
prompt_template = """
<s>[INST] 

Assistant is a large language model trained by Mistral.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

Context:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action , please give full details of the result of the tool call
'''

When you have a response to say to the Human,  you MUST use the format:

'''
Final Answer: [your response here]
'''

Here is the query from the human:
{input}

Current Scratchpad:
{agent_scratchpad}

Now please think and answer : 

[/INST]
 """

In [15]:
from langchain.tools.render import render_text_description
from langchain_core.prompts import PromptTemplate
from langchain.agents.format_scratchpad import format_log_to_str

In [16]:
tools = [search ,Gog_tool , Buy_tool]

In [17]:
prompt = PromptTemplate(
    input_variables=['agent_scratchpad',  'input', 'tool_names', 'tools'],
    template=prompt_template,
)

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)



from typing import List, Union

In [18]:
from langchain.agents import AgentOutputParser
from langchain_core.agents import AgentAction , AgentFinish
from langchain_core.exceptions import OutputParserException

from typing import List, Union

class CustomOutputParser(AgentOutputParser):
    def parse(self, output) -> Union[AgentAction, AgentFinish]:
        print('text :',output)
        output = ''.join(output.split("Now please think and answer : ")[1]).strip()
        print('output : ', output)
        if 'Final Answer:' in output:
            final_answer = output.split('Final Answer:')[1].strip()
            agent_finish = AgentFinish(
                return_values={"output": final_answer},
                log=output,
            )
            return agent_finish
        else:
            # Extract the tool and action input from the output
            tool = output.split('Action:')[1].split('Action Input:')[0].strip()
            print('tool : ', tool)
            action_input = output.split('Action Input:')[1].strip()
            print('action input : ', action_input)
            agent_action = AgentAction(
                tool=tool,
                action_input=action_input,
                log=output,
            )
            return agent_action

In [19]:
from langchain.agents import AgentExecutor

output_parser = CustomOutputParser()

agent =({
    "input" : lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])}
    
    | prompt
    | Ch
    | output_parser
)

agent_executor = AgentExecutor(agent = agent 
                               , tools = tools,
                               verbose = True)

query ="what is the price of AXA stock (FRA:AXA)? Please return the Open price of the stock today at the Paris place.If the price is below 100 euros please buy 5 shares using the Buy tool"

agent_executor.invoke({"input" : query})



> Entering new AgentExecutor chain...


c:\Users\benya\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


text : <s> [INST] 
<s>[INST] 

Assistant is a large language model trained by Mistral.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can 

{'input': 'what is the price of AXA stock (FRA:AXA)? Please return the Open price of the stock today at the Paris place.If the price is below 100 euros please buy 5 shares using the Buy tool',
 'output': 'The price of AXA stock (FRA:AXA) at the Paris Bourse today is 115.25 euros.'}

In [20]:
torch.cuda.current_device()



0

In [21]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Laptop GPU'